In [1]:
from biodatasets import list_datasets, load_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

In [2]:
# Loading data into numpy array
pathogen = load_dataset("pathogen")

X, y = pathogen.to_npy_arrays(input_names=["sequence"], target_names=["class"])

pathogen.display_description()

# "Human vs pathogen" dataset


## Dataset Description
Human vs pathogen clasification dataset.

### Dataset Summary

Human vs pathogen clasification dataset. 96k protein sequences (50% human, 50% pathogens). Extracted from Uniprot. Embeddings available calculated with ProtBert

Features:
 - sequence
 - sequence_id

Embeddings:
 - CLS embeddings - 1024-dim

Label:
 - class
  - 0: human
  - 1: pathogen

### Usage
```
from biodatasets import load_dataset

pathogen_dataset = load_dataset("pathogen")

X, y = pathogen_dataset.to_npy_array(input_names=["sequence"], target_names=["class"])
cls_embeddings = pathogen_dataset.get_embeddings("sequence", "protbert", "cls")
```

### Supported Tasks
 - clasification
 - inmunogenecity

### Model used to calculate Embeddings
 - ProtBert

### Libraries used to calculate embeddings
 - Pytorch


### Source Data

[Uniprot](https://www.uniprot.org/)


### Dataset Curators

[DeepChain team](https://deepchain.bio)

### Licensing Information
[Creative Commons

In [3]:
# Encoding Amino Acids to number
def get_seq_column_map(X):
    unique = set()
    for idx, sequence in enumerate(X[0]):
        unique.update(list(sequence))
    
    return dict(zip(unique, list(range(len(unique)))))
    
pathogen_map = get_seq_column_map(X)
print(pathogen_map)

{'P': 0, 'N': 1, 'K': 2, 'I': 3, 'R': 4, 'U': 5, 'D': 6, 'M': 7, 'Y': 8, 'F': 9, 'X': 10, 'L': 11, 'A': 12, 'B': 13, 'E': 14, 'C': 15, 'S': 16, 'W': 17, 'V': 18, 'T': 19, 'Q': 20, 'H': 21, 'Z': 22, 'G': 23}


In [4]:
class PathogenDataset(Dataset):
    
    def __init__(self, pathogen_map, data):
        self.pathogen_map = pathogen_map
        self.X = data[0]
        self.Y = data[1]
        
    def __one_hot(self, Y):
        one_hot_Y = np.zeros((Y.size, Y.max() + 1))
        one_hot_Y[np.arange(Y.size), Y] = 1
        one_hot_Y = one_hot_Y
        return one_hot_Y.astype(np.float64)
    
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        X = torch.as_tensor([self.pathogen_map[e] for e in list(self.X[idx])]) 
        Y = self.Y[idx]
        return X, Y

def collate_padd(batch):
        x = [row[0] for row in batch]
        y = [row[1] for row in batch]
        
        sequence_len = [len(row) for row in x]
        x =  pad_sequence(x, batch_first=True)
        return (torch.as_tensor(x).to(torch.float32), torch.as_tensor(sequence_len)), torch.as_tensor(y).to(torch.float32)
    
# Split ~ 80% 10% 10%
training_set = PathogenDataset(pathogen_map,(X[0][:80000], y[0][:80000]))
training_loader = DataLoader(training_set, batch_size=8, shuffle=True, collate_fn=collate_padd)

validation_set = PathogenDataset(pathogen_map,(X[0][80000:90000], y[0][80000:90000]))
validation_loader = DataLoader(validation_set, batch_size=8, collate_fn=collate_padd)

testing_set = PathogenDataset(pathogen_map,(X[0][90000:], y[0][90000:]))
testing_loader = DataLoader(testing_set, batch_size=8, collate_fn=collate_padd)

next(iter(training_loader))

((tensor([[11., 16., 11.,  ...,  0.,  0.,  0.],
          [ 7.,  0., 21.,  ...,  0.,  0.,  0.],
          [ 7., 12., 16.,  ...,  0.,  0.,  0.],
          ...,
          [11.,  2., 14.,  ...,  0.,  0.,  0.],
          [ 7.,  2., 11.,  ...,  0.,  0.,  0.],
          [ 2., 19., 19.,  ...,  0.,  0.,  0.]]),
  tensor([  41,  128,  400,   38, 1090,  248,  474,   57])),
 tensor([0., 0., 0., 1., 0., 0., 1., 0.]))

In [8]:
class Net(nn.Module):
    
    def __init__(self, input_dim=len(pathogen_map)):
        super().__init__()
        
        self.embed = nn.Embedding(
            num_embeddings=input_dim,
            embedding_dim=512,
        )
        
        self.lstm = nn.LSTM(
            input_size=512,
            hidden_size=256,
            num_layers=1,
            batch_first=True,
        )
        
        self.linear_1 = nn.Linear(
            in_features=256,
            out_features=128,
        )
        
        self.dropout = nn.Dropout(p=0.25)
        
        self.linear_2 = nn.Linear(
            in_features=128,
            out_features=1,
        )
        
    def forward(self, x, sequence_len):  
        embed = self.embed(x)
        
        packed_input = pack_padded_sequence(embed, sequence_len, batch_first=True, enforce_sorted=False)
        lstm_1_seq, _ = self.lstm(packed_input)
        output, _ = pad_packed_sequence(lstm_1_seq, batch_first=True)
        
        out_forward = output[range(len(output)), sequence_len - 1]
        
        dropout = self.dropout(out_forward)
        
        linear_1 = self.linear_1(dropout)
        dropout = self.dropout(linear_1)
        linear_2 = self.linear_2(dropout)
        
        return torch.squeeze(linear_2)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device : {device}")
torch.cuda.get_device_name()

device : cuda


'NVIDIA GeForce RTX 2060 SUPER'

In [ ]:
model = Net().cuda()
print(model)

writer = SummaryWriter()

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(2):
    model.train()
    tqdm_bar = tqdm(training_loader, desc=f"epoch {epoch}", position=0)
    
    # Training
    for idx, ((inputs, sequence_len), labels) in enumerate(tqdm_bar):
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        optimizer.zero_grad()
        
        outputs = model(inputs.to(torch.int32), sequence_len)
               
        loss = criterion(outputs, labels).to(torch.float32)
        loss.backward()
        
        writer.add_scalar('Loss/train', loss, idx)
        
        
        # Training Accuracy
        correct, total = 0, 0
        predicted = torch.round(outputs.flatten())
        y = labels

        total += labels.size(0)
        correct += (predicted == y).sum().item()
        writer.add_scalar('accuracy/train', correct/total, idx)
        
        optimizer.step()
        
    
    # Validation Accuracy
    model.eval()
    with torch.no_grad():
        for idx, ((inputs, sequence_len), labels)  in enumerate(validation_loader):
            correct, total = 0, 0
            inputs = inputs.cuda()
            labels = labels.cuda()

            outputs = model(inputs.to(torch.int32), sequence_len)

            predicted = torch.round(outputs.flatten())
            y = labels

            total += labels.size(0)
            correct += (predicted == y).sum().item()
            writer.add_scalar('accuracy/validation', correct/total, idx)
    
writer.close()

epoch 0:   0%|                                | 3/10000 [00:00<08:21, 19.95it/s]

Net(
  (embed): Embedding(24, 512)
  (lstm): LSTM(512, 256, batch_first=True)
  (linear_1): Linear(in_features=256, out_features=128, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (linear_2): Linear(in_features=128, out_features=1, bias=True)
)


epoch 0:   2%|▋                             | 237/10000 [00:09<06:28, 25.11it/s]

In [18]:
!tensorboard --logdir=runs

2022-07-20 13:27:47.766564: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-20 13:27:47.766584: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-20 13:27:48.819478: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 13:27:48.819805: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-20 13:27:48.819844: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s

In [11]:
PATH = './pathogen_net.pth'
torch.save(model.state_dict(), PATH)

In [14]:
model = Net().cuda()
model.load_state_dict(torch.load(PATH))
model.eval()


# Testing Accuracy
correct, total = 0, 0
with torch.no_grad():
    for ((inputs, sequence_len), labels) in testing_loader:
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        outputs = model(inputs.to(torch.int32), sequence_len)
        
        predicted = torch.round(outputs.flatten())
        y = labels
        
        print(predicted, y)
        
        total += labels.size(0)
        correct += (predicted == y).sum().item()
        
print(f'Accuracy of nn: {correct / total}')

tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 1., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., 0., -0.], device='cuda:0') tensor([1., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., -0., -0., -0., -0., 0., 0., -0.], device='cuda:0') tensor([1., 0., 0., 0., 1., 1., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 1., 0., 1.], device='cuda:0')
tensor([-0., -0., -0., -0., 0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 1., 1., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., 0., -0., -0.], device='cuda:0') tensor([1., 0., 0., 0., 0., 1., 1., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 1., 0., 0., 0.], device='cuda:0')
tensor([0., 0., -0., -0., -0., -0., -0., 0.], device='cuda:0') tensor([1., 0., 0., 1., 0., 1., 0., 0.], device='cuda:0')
tensor([-0., -0., 

tensor([-0., -0., -0., -0., -0., -0., -0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., 0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0

tensor([0., 0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., 0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., 0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0.,

tensor([0., 0., 0., -0., 0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., -0., 0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., -0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., -0., 0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., -0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., -0., -0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., -0., 0., 0., -0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., -0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., 0., 0., 0., 0., 0., 0., -0.], device='cuda:0') te

tensor([0., 0., 0., 0., -0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., -0., 0., -0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., -0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., -0., 0., 0., -0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., -0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor(

tensor([-0., -0., -0., -0., -0., 0., 0., 0.], device='cuda:0') tensor([1., 1., 0., 0., 1., 1., 1., 1.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([1., 0., 0., 0., 0., 0., 0., 1.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., 0., -0.], device='cuda:0') tensor([0., 1., 1., 1., 1., 1., 0., 0.], device='cuda:0')
tensor([-0., 0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., 0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 1., 0., 1.], device='cuda:0')
tensor([-0., 0., 0., -0., 0., -0., -0., -0.], device='cuda:0') tensor([0., 1., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., 0., -0., -0., -0., 0.], device='cuda:0') tensor([0., 1., 0., 0., 0., 0., 1., 0.], device='cuda:0')
tensor([0., 0., -0., -0., 0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 1., 0., 1., 0., 0.], device='cuda:0')
tensor([-0., -0., -0.,

tensor([-0., -0., -0., -0., -0., -0., 0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., 0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., 0., -0., -0., 0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., -0., 0., 0., -0., 0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., 0., 0., -0., 0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., 0

tensor([-0., -0., -0., 0., 0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., 0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0.,

tensor([-0., -0., -0., 0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., -0., 0., 0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., 0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., 0., -0., -0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., -0., -0., -0., -0., -0., -0., 0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., 

tensor([0., 0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., 0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 1., 1., 0., 1., 0., 0., 1.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 1., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., 0.], device='cuda:0') tensor([0., 1., 0., 1., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., 0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., 0., -0., -0., -0., 0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 1., 1., 1., 1.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., 0., -0., -0.], device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-0., -0., -0., -0., -0., -0., -0., -0.], device='cuda:0') tensor([0., 1., 0., 0., 0., 0., 1., 0.], device='cuda:0')
tensor([-0., -0.